In [147]:
import os.path
import glob
import csv
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import dask.dataframe as dd
from dask.distributed import Client
from dask.distributed import get_worker
from fabric import Connection, Config

In [148]:
client = Client("tcp://128.110.217.26:8786")

In [149]:
client

<Client: 'tcp://128.110.217.26:8786' processes=2 threads=32, memory=125.38 GiB>

In [150]:
def create_metadata():
    mnist_data_path = "/users/prsridha/data/trainingSet/trainingSet/"
    mnist_metadata_out_path = "/users/prsridha/data/metadata.csv"
    data_rows = []
    num_labels = 10
    for i in range(num_labels):
        curr_label_path = mnist_data_path + str(i) + "/"
        jpgFilenamesList =  glob.glob(curr_label_path + '*.jpg')
        rows = [[str(fname), i] for fname in jpgFilenamesList]
        data_rows.extend(rows)
        # print(curr_label_path, str(len(jpgFilenamesList)))
    with open(mnist_metadata_out_path, 'w') as f: 
        write = csv.writer(f)
        write.writerows(data_rows)

In [151]:
create_metadata()

In [152]:
DATA_URL = "/users/prsridha/data/metadata.csv"
feature_names = ["image_path", "label"]
dtypes = {'image_path': str, 'label': np.int16}
df0 = pd.read_csv(DATA_URL, names=feature_names, dtype=dtypes)
df0 = dd.from_pandas(df0, npartitions=2)
df = df0.sample(frac=0.001)

In [153]:
df.head()

,image_path,label
11194,/users/prsridha/data/trainingSet/trainingSet/2...,2
2046,/users/prsridha/data/trainingSet/trainingSet/0...,0
20016,/users/prsridha/data/trainingSet/trainingSet/4...,4
11180,/users/prsridha/data/trainingSet/trainingSet/2...,2
9531,/users/prsridha/data/trainingSet/trainingSet/2...,2


In [154]:
shuffled_df = df.sample(frac=1)
shuffled_df.head()

,image_path,label
19141,/users/prsridha/data/trainingSet/trainingSet/4...,4
9531,/users/prsridha/data/trainingSet/trainingSet/2...,2
761,/users/prsridha/data/trainingSet/trainingSet/0...,0
4623,/users/prsridha/data/trainingSet/trainingSet/1...,1
2046,/users/prsridha/data/trainingSet/trainingSet/0...,0


In [155]:
new_df = shuffled_df.repartition(npartitions=2)

In [156]:
user = "prsridha"
host = "128.110.217.26"
pem_path = "./cloudlab.pem"
connect_kwargs = {"key_filename":pem_path}
conn = Connection(host, user=user, connect_kwargs=connect_kwargs)

In [157]:
def pull(from_path, to_path, conn):
    to_path_dir = "/".join(to_path.split("/")[:-1])
    Path(to_path_dir).mkdir(parents=True, exist_ok=True)
    
    result = conn.get(from_path, to_path)
    print("Pulled from {0.remote} to {0.local}".format(result))

In [159]:
def resize_image(path, conn):
    if not os.path.isfile(path):
        pull(path, path, conn)
    
    im = Image.open(path)
    resized = im.resize((14,14))
    resized_np = np.array(resized)
    return resized_np

In [160]:
new_col = new_df.image_path.map_partitions(lambda x: x.apply(lambda y: resize_image(y, conn)), meta=('new_col', object))

In [161]:
new_col.compute()

19141    [[0, 0, 0, 0, 1, 0, 0, 0, 3, 6, 0, 2, 0, 0], [...
9531     [[0, 1, 1, 2, 3, 0, 2, 3, 2, 2, 1, 3, 0, 0], [...
761      [[0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 2, 0, 0], [...
4623     [[0, 0, 0, 0, 1, 1, 2, 1, 1, 0, 1, 2, 0, 0], [...
2046     [[0, 0, 0, 0, 1, 3, 4, 2, 0, 1, 3, 4, 0, 0], [...
18698    [[0, 0, 0, 0, 2, 2, 2, 3, 2, 1, 2, 2, 0, 0], [...
3907     [[0, 0, 0, 0, 1, 2, 2, 3, 3, 1, 0, 2, 0, 0], [...
16464    [[0, 0, 0, 0, 2, 3, 2, 0, 3, 0, 1, 5, 0, 0], [...
7781     [[0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 3, 2, 0, 0], [...
5140     [[0, 0, 0, 0, 1, 0, 0, 0, 2, 3, 6, 4, 1, 0], [...
11194    [[3, 2, 5, 0, 0, 0, 0, 1, 3, 3, 2, 2, 0, 0], [...
11180    [[0, 0, 0, 0, 2, 4, 0, 2, 0, 0, 2, 1, 0, 0], [...
20016    [[0, 0, 0, 0, 5, 3, 2, 0, 0, 0, 0, 0, 0, 0], [...
6466     [[0, 0, 0, 0, 1, 3, 0, 1, 1, 1, 1, 2, 0, 0], [...
5856     [[0, 0, 0, 0, 3, 2, 0, 3, 1, 0, 0, 0, 0, 0], [...
1589     [[0, 0, 0, 0, 1, 1, 2, 3, 2, 1, 2, 2, 0, 1], [...
12687    [[0, 0, 0, 0, 3, 4, 1, 0, 0, 0, 2, 3, 1, 0], [.

In [214]:
x = list(new_col.iteritems())[0]
im = Image.fromarray(x[1])
im.show()

In [215]:
y = list(new_df.image_path.iteritems())[0]
im1 = Image.open(y[1])
im1.show()

In [216]:
conn.close()